<h2>Import Libraries</h2>

In [1]:
import datetime
from dateutil import tz

import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar as fedcalendar

from suntime import Sun

<h2>Import Data</h2>

In [2]:
df_weather = pd.read_parquet('data/weather.parquet')
df_weather.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH
1,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049
2,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603
3,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167
4,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391
5,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103


In [3]:
df_census = pd.read_csv('data/United States-.csv', skiprows=8)
df_census.head()

,Date,GDP (Billions of US $),Per Capita (US $),Annual % Change
0,1950-12-31,4999000,NaN,NaN
1,1951-12-31,5107000,2.16,NaN
2,1952-12-31,5218000,2.17,NaN
3,1953-12-31,5331000,2.17,NaN
4,1954-12-31,5447000,2.18,NaN


In [4]:
df_power = pd.read_csv('data/COMED_hourly.csv')
df_power.head()

,Datetime,COMED_MW
0,2011-12-31 01:00:00,9970.0
1,2011-12-31 02:00:00,9428.0
2,2011-12-31 03:00:00,9059.0
3,2011-12-31 04:00:00,8817.0
4,2011-12-31 05:00:00,8743.0


<h2>Clean Data - Census</h2>

In [5]:
# remove funky first entry - don't care about 1950 data
df_census = df_census[df_census['Date']!='1950-12-31']

In [6]:
# get year from date
df_census['Date'] = pd.to_datetime(df_census['Date'], utc=True)
df_census['YEAR'] = df_census['Date'].dt.year
df_census.head()

,Date,GDP (Billions of US $),Per Capita (US $),Annual % Change,YEAR
1,1951-12-31 00:00:00+00:00,5107000,2.16,NaN,1951
2,1952-12-31 00:00:00+00:00,5218000,2.17,NaN,1952
3,1953-12-31 00:00:00+00:00,5331000,2.17,NaN,1953
4,1954-12-31 00:00:00+00:00,5447000,2.18,NaN,1954
5,1955-12-31 00:00:00+00:00,5565000,2.17,NaN,1955


In [7]:
# rename columns
df_census = df_census.rename(columns={' GDP (Billions of US $)': 'GDP', 
                                      ' Per Capita (US $)': 'per capita'})
df_census.head()

,Date,GDP,per capita,Annual % Change,YEAR
1,1951-12-31 00:00:00+00:00,5107000,2.16,NaN,1951
2,1952-12-31 00:00:00+00:00,5218000,2.17,NaN,1952
3,1953-12-31 00:00:00+00:00,5331000,2.17,NaN,1953
4,1954-12-31 00:00:00+00:00,5447000,2.18,NaN,1954
5,1955-12-31 00:00:00+00:00,5565000,2.17,NaN,1955


In [8]:
# add population
# rounding to nearest thousand so more useful in the future - won't necessarily know population to nearest person
df_census['POPULATION'] = round(df_census['GDP']/df_census['per capita'], -3).astype(int)
df_census.head()

,Date,GDP,per capita,Annual % Change,YEAR,POPULATION
1,1951-12-31 00:00:00+00:00,5107000,2.16,NaN,1951,2364000
2,1952-12-31 00:00:00+00:00,5218000,2.17,NaN,1952,2405000
3,1953-12-31 00:00:00+00:00,5331000,2.17,NaN,1953,2457000
4,1954-12-31 00:00:00+00:00,5447000,2.18,NaN,1954,2499000
5,1955-12-31 00:00:00+00:00,5565000,2.17,NaN,1955,2565000


In [9]:
df_census.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 1 to 85
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Date              85 non-null     datetime64[ns, UTC]
 1   GDP               85 non-null     int64              
 2   per capita        85 non-null     float64            
 3    Annual % Change  0 non-null      float64            
 4   YEAR              85 non-null     int32              
 5   POPULATION        85 non-null     int32              
dtypes: datetime64[ns, UTC](1), float64(2), int32(2), int64(1)
memory usage: 4.0 KB


In [10]:
# drop extra columns
df_census = df_census.drop(columns=['Date', ' Annual % Change', 'per capita'])
df_census.head()

,GDP,YEAR,POPULATION
1,5107000,1951,2364000
2,5218000,1952,2405000
3,5331000,1953,2457000
4,5447000,1954,2499000
5,5565000,1955,2565000


<h2>Clean Data - Power</h2>

In [11]:
# make datetime actually datetime
df_power['Datetime'] = pd.to_datetime(df_power['Datetime'])
df_power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66497 entries, 0 to 66496
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  66497 non-null  datetime64[ns]
 1   COMED_MW  66497 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.0 MB


In [12]:
# rename to stay consistent with weather data
df_power = df_power.rename(columns={'Datetime': 'DATE'})
df_power.head()

,DATE,COMED_MW
0,2011-12-31 01:00:00,9970.0
1,2011-12-31 02:00:00,9428.0
2,2011-12-31 03:00:00,9059.0
3,2011-12-31 04:00:00,8817.0
4,2011-12-31 05:00:00,8743.0


In [13]:
# convert to UTC
# kept getting errors, so doing this a hard way
df_power['DATE_temp'] = df_power['DATE'].dt.tz_localize('America/Chicago', ambiguous='NaT', nonexistent='NaT')

# separate out the good values and bad
df_power_bad = df_power[df_power['DATE_temp'].isna()].copy()
df_power_good = df_power[~df_power['DATE_temp'].isna()].copy()

df_power_bad

,DATE,COMED_MW,DATE_temp
1320,2011-11-06 01:00:00,9164.0,NaT
7032,2011-03-13 02:00:00,9719.0,NaT
10126,2012-11-04 01:00:00,9382.0,NaT
15838,2012-03-11 02:00:00,8773.0,NaT
18932,2013-11-03 01:00:00,9230.0,NaT
24644,2013-03-10 02:00:00,9334.0,NaT
27714,2014-11-02 01:00:00,9573.0,NaT
33428,2014-03-09 02:00:00,10268.0,NaT
36498,2015-11-01 01:00:00,8663.0,NaT
42212,2015-03-08 02:00:00,9433.0,NaT


In [14]:
# fix bad values
df_power_bad['DATE_temp'] = df_power_bad['DATE'].dt.tz_localize('America/Chicago', 
                                                                ambiguous=np.array([True, False, 
                                                                                    True, False, 
                                                                                    True, False,
                                                                                    True, False, 
                                                                                    True, False,
                                                                                    True, False,
                                                                                    True, False, False]), 
                                                                nonexistent='shift_forward')

df_power_bad

,DATE,COMED_MW,DATE_temp
1320,2011-11-06 01:00:00,9164.0,2011-11-06 01:00:00-05:00
7032,2011-03-13 02:00:00,9719.0,2011-03-13 03:00:00-05:00
10126,2012-11-04 01:00:00,9382.0,2012-11-04 01:00:00-05:00
15838,2012-03-11 02:00:00,8773.0,2012-03-11 03:00:00-05:00
18932,2013-11-03 01:00:00,9230.0,2013-11-03 01:00:00-05:00
24644,2013-03-10 02:00:00,9334.0,2013-03-10 03:00:00-05:00
27714,2014-11-02 01:00:00,9573.0,2014-11-02 01:00:00-05:00
33428,2014-03-09 02:00:00,10268.0,2014-03-09 03:00:00-05:00
36498,2015-11-01 01:00:00,8663.0,2015-11-01 01:00:00-05:00
42212,2015-03-08 02:00:00,9433.0,2015-03-08 03:00:00-05:00


In [15]:
# combine and swap to UTC
df_power = pd.concat([df_power_good, df_power_bad])
df_power['DATE'] = df_power['DATE_temp'].dt.tz_convert('UTC')
df_power = df_power.drop(columns=['DATE_temp'])

df_power.head()

,DATE,COMED_MW
0,2011-12-31 07:00:00+00:00,9970.0
1,2011-12-31 08:00:00+00:00,9428.0
2,2011-12-31 09:00:00+00:00,9059.0
3,2011-12-31 10:00:00+00:00,8817.0
4,2011-12-31 11:00:00+00:00,8743.0


In [16]:
# drop duplicates
df_power = df_power.drop_duplicates(subset=['DATE'])
df_power.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66493 entries, 0 to 64819
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   DATE      66493 non-null  datetime64[ns, UTC]
 1   COMED_MW  66493 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 1.5 MB


<h2>Join Dateframes</h2>

In [17]:
df = pd.merge(df_weather, df_power, on='DATE', how='inner')
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0


In [18]:
df['YEAR'] = df['DATE'].dt.year
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017


In [19]:
df = pd.merge(df, df_census, on='YEAR', how='inner')
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000


<h2>Add potentially helpful columns</h2>

In [20]:
# add adjusted for time zone date column
df['DATE_CENTRAL'] = df['DATE'].dt.tz_convert('America/Chicago')
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00


In [21]:
# add general helpful date columns
df_weekday = pd.get_dummies(df['DATE_CENTRAL'].dt.day_name(), prefix='WEEKDAY')
df_month = pd.get_dummies(df['DATE_CENTRAL'].dt.month, prefix='MONTH')
df = pd.concat([df, df_weekday, df_month], axis=1)

df['HOUR'] = df['DATE_CENTRAL'].dt.hour
df['MONTHDAY'] = df['DATE_CENTRAL'].dt.day
df['DAYOFYEAR'] = df['DATE_CENTRAL'].dt.day_of_year
df['MONTHDAYRATIO'] = df['MONTHDAY']/df['DATE_CENTRAL'].dt.days_in_month
df['IS_WEEKEND'] = df['DATE_CENTRAL'].dt.dayofweek >= 5
df['IS_DAYLIGHTSAVINGS'] = df['DATE_CENTRAL'].dt.strftime('%z')=='-0500'

df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,MONTH_9,MONTH_10,MONTH_11,MONTH_12,HOUR,MONTHDAY,DAYOFYEAR,MONTHDAYRATIO,IS_WEEKEND,IS_DAYLIGHTSAVINGS
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,False,False,False,True,19,31,366,1.0,True,False
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,False,False,False,True,20,31,366,1.0,True,False
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,False,False,False,True,21,31,366,1.0,True,False
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,False,False,False,True,22,31,366,1.0,True,False
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,False,False,False,True,23,31,366,1.0,True,False


In [22]:
# add seasons
df['MONTH'] = df['DATE_CENTRAL'].dt.month
df['SEASON'] = df['MONTH'].map({12: 'WINTER', 1:'WINTER', 2:'WINTER', 
                                3:'SPRING', 4:'SPRING', 5:'SPRING', 
                                6:'SUMMER', 7:'SUMMER', 8:'SUMMER',
                                9:'FALL', 10:'FALL', 11:'FALL'})
df_seasons = pd.get_dummies(df['SEASON'])

df = pd.concat([df, df_seasons], axis=1)

# leaving season for visualizations
df = df.drop(columns=['MONTH'])

df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,MONTHDAY,DAYOFYEAR,MONTHDAYRATIO,IS_WEEKEND,IS_DAYLIGHTSAVINGS,SEASON,FALL,SPRING,SUMMER,WINTER
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,31,366,1.0,True,False,WINTER,False,False,False,True
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,31,366,1.0,True,False,WINTER,False,False,False,True
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,31,366,1.0,True,False,WINTER,False,False,False,True
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,31,366,1.0,True,False,WINTER,False,False,False,True
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,31,366,1.0,True,False,WINTER,False,False,False,True


In [23]:
# add epoch time - gets numeric for later processing
df['epoch'] = df['DATE'].astype('int64')//1e9
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,DAYOFYEAR,MONTHDAYRATIO,IS_WEEKEND,IS_DAYLIGHTSAVINGS,SEASON,FALL,SPRING,SUMMER,WINTER,epoch
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,366,1.0,True,False,WINTER,False,False,False,True,1.483232e+09
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,366,1.0,True,False,WINTER,False,False,False,True,1.483236e+09
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,366,1.0,True,False,WINTER,False,False,False,True,1.483240e+09
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,366,1.0,True,False,WINTER,False,False,False,True,1.483243e+09
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,366,1.0,True,False,WINTER,False,False,False,True,1.483247e+09


In [24]:
# add markers for holidays
fedcal = fedcalendar()
holidays_list = fedcal.holidays()
holidays_list = [i.tz_localize('America/Chicago') for i in holidays_list if i.year>=2010 and i.year<=2018]

df['IS_HOLIDAY'] = df['DATE_CENTRAL'].dt.normalize().isin(holidays_list)
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,MONTHDAYRATIO,IS_WEEKEND,IS_DAYLIGHTSAVINGS,SEASON,FALL,SPRING,SUMMER,WINTER,epoch,IS_HOLIDAY
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,1.0,True,False,WINTER,False,False,False,True,1.483232e+09,False
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,1.0,True,False,WINTER,False,False,False,True,1.483236e+09,False
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,1.0,True,False,WINTER,False,False,False,True,1.483240e+09,False
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,1.0,True,False,WINTER,False,False,False,True,1.483243e+09,False
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,1.0,True,False,WINTER,False,False,False,True,1.483247e+09,False


In [25]:
# add markers for near holidays
next_day = [i+datetime.timedelta(days=1) for i in holidays_list]
day_before = [i-datetime.timedelta(days=1) for i in holidays_list]

next_day2 = [i+datetime.timedelta(days=2) for i in holidays_list]
day_before2 = [i-datetime.timedelta(days=2) for i in holidays_list]

df['NEAR_HOLIDAY'] = (df['DATE_CENTRAL'].dt.normalize().isin(next_day))|(df['DATE_CENTRAL'].dt.normalize().isin(day_before))
df['2offHOLIDAY'] = (df['DATE_CENTRAL'].dt.normalize().isin(next_day2))|(df['DATE_CENTRAL'].dt.normalize().isin(day_before2))
df['HOLIDAY_WEEKEND'] = df['IS_WEEKEND'] & (df['IS_HOLIDAY']|df['NEAR_HOLIDAY']|df['2offHOLIDAY'])

df = df.drop(columns=['2offHOLIDAY'])

df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,IS_DAYLIGHTSAVINGS,SEASON,FALL,SPRING,SUMMER,WINTER,epoch,IS_HOLIDAY,NEAR_HOLIDAY,HOLIDAY_WEEKEND
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,False,WINTER,False,False,False,True,1.483232e+09,False,False,True
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,False,WINTER,False,False,False,True,1.483236e+09,False,False,True
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,False,WINTER,False,False,False,True,1.483240e+09,False,False,True
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,False,WINTER,False,False,False,True,1.483243e+09,False,False,True
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,False,WINTER,False,False,False,True,1.483247e+09,False,False,True


In [26]:
# add markers for time of day
# Chicago: 41.8781° N, 87.6298° W
chicago_lat = 41.8781
chicago_lon = -87.6298

sun = Sun(chicago_lat, chicago_lon)

df['SUNRISE'] = pd.to_datetime(df['DATE'].apply(lambda x: sun.get_sunrise_time(at_date=x)))
df['SUNSET'] = pd.to_datetime(df['DATE'].apply(lambda x: sun.get_sunset_time(at_date=x)))

df['IS_DAYTIME'] = (df['DATE']>=df['SUNRISE'])&(df['DATE']<df['SUNSET'])
df['DAWN'] = (df['DATE']>=(df['SUNRISE']-datetime.timedelta(hours=1)))&(df['DATE']<=(df['SUNRISE']+datetime.timedelta(hours=1)))
df['DUSK'] = (df['DATE']>=(df['SUNSET']-datetime.timedelta(hours=1)))&(df['DATE']<=(df['SUNSET']+datetime.timedelta(hours=1)))

df = df.drop(columns=['SUNRISE', 'SUNSET'])

df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,SPRING,SUMMER,WINTER,epoch,IS_HOLIDAY,NEAR_HOLIDAY,HOLIDAY_WEEKEND,IS_DAYTIME,DAWN,DUSK
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,False,False,True,1.483232e+09,False,False,True,False,False,False
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,False,False,True,1.483236e+09,False,False,True,False,False,False
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,False,False,True,1.483240e+09,False,False,True,False,False,False
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,False,False,True,1.483243e+09,False,False,True,False,False,False
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,False,False,True,1.483247e+09,False,False,True,False,False,False


In [27]:
df[df['IS_HOLIDAY']]

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,SPRING,SUMMER,WINTER,epoch,IS_HOLIDAY,NEAR_HOLIDAY,HOLIDAY_WEEKEND,IS_DAYTIME,DAWN,DUSK
23,2017-01-02 06:00:00+00:00,28.94,50,5.81594,81.806302,10114.0,2017,8833000,24536000,2017-01-02 00:00:00-06:00,...,False,False,True,1.483337e+09,True,False,False,False,False,False
24,2017-01-02 07:00:00+00:00,30.02,50,4.69749,78.283528,9678.0,2017,8833000,24536000,2017-01-02 01:00:00-06:00,...,False,False,True,1.483340e+09,True,False,False,False,False,False
25,2017-01-02 08:00:00+00:00,28.04,30,3.35535,84.876963,9296.0,2017,8833000,24536000,2017-01-02 02:00:00-06:00,...,False,False,True,1.483344e+09,True,False,False,False,False,False
26,2017-01-02 09:00:00+00:00,28.94,40,5.81594,84.934853,9076.0,2017,8833000,24536000,2017-01-02 03:00:00-06:00,...,False,False,True,1.483348e+09,True,False,False,False,False,False
27,2017-01-02 10:00:00+00:00,28.94,40,6.93439,88.829090,8995.0,2017,8833000,24536000,2017-01-02 04:00:00-06:00,...,False,False,True,1.483351e+09,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56612,2012-12-26 01:00:00+00:00,28.04,360,6.93439,84.876963,11354.0,2012,8677000,24103000,2012-12-25 19:00:00-06:00,...,False,False,True,1.356484e+09,True,False,False,False,False,False
56613,2012-12-26 02:00:00+00:00,28.04,360,5.81594,84.876963,11309.0,2012,8677000,24103000,2012-12-25 20:00:00-06:00,...,False,False,True,1.356487e+09,True,False,False,False,False,False
56614,2012-12-26 03:00:00+00:00,28.94,350,6.93439,84.934853,11247.0,2012,8677000,24103000,2012-12-25 21:00:00-06:00,...,False,False,True,1.356491e+09,True,False,False,False,False,False
56615,2012-12-26 04:00:00+00:00,32.00,60,10.28974,72.255158,11198.0,2012,8677000,24103000,2012-12-25 22:00:00-06:00,...,False,False,True,1.356494e+09,True,False,False,False,False,False


In [28]:
# add min/max temperature for the day
df['ROUNDED_DAY'] = df['DATE_CENTRAL'].dt.normalize()
df_minmax_temps = df[['ROUNDED_DAY', 'TMP']].groupby(by='ROUNDED_DAY', as_index=False).agg({'TMP': ['min', 'max']})
df_minmax_temps.columns = ['ROUNDED_DAY', 'MIN_TEMP', 'MAX_TEMP']

df = pd.merge(df, df_minmax_temps, on='ROUNDED_DAY')
df = df.drop(columns=['ROUNDED_DAY'])
df.head()

,DATE,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,DATE_CENTRAL,...,WINTER,epoch,IS_HOLIDAY,NEAR_HOLIDAY,HOLIDAY_WEEKEND,IS_DAYTIME,DAWN,DUSK,MIN_TEMP,MAX_TEMP
0,2017-01-01 01:00:00+00:00,28.04,260,6.93439,71.273049,11587.0,2017,8833000,24536000,2016-12-31 19:00:00-06:00,...,True,1.483232e+09,False,False,True,False,False,False,24.98,39.92
1,2017-01-01 02:00:00+00:00,26.96,260,4.69749,71.701603,11549.0,2017,8833000,24536000,2016-12-31 20:00:00-06:00,...,True,1.483236e+09,False,False,True,False,False,False,24.98,39.92
2,2017-01-01 03:00:00+00:00,26.06,250,5.81594,74.418167,11273.0,2017,8833000,24536000,2016-12-31 21:00:00-06:00,...,True,1.483240e+09,False,False,True,False,False,False,24.98,39.92
3,2017-01-01 04:00:00+00:00,26.06,240,6.93439,77.333391,11085.0,2017,8833000,24536000,2016-12-31 22:00:00-06:00,...,True,1.483243e+09,False,False,True,False,False,False,24.98,39.92
4,2017-01-01 05:00:00+00:00,24.98,250,6.93439,80.879103,10801.0,2017,8833000,24536000,2016-12-31 23:00:00-06:00,...,True,1.483247e+09,False,False,True,False,False,False,24.98,39.92


<h2>Final Dataset</h2>

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56746 entries, 0 to 56745
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype                          
---  ------              --------------  -----                          
 0   DATE                56746 non-null  datetime64[ns, UTC]            
 1   TMP                 56746 non-null  float64                        
 2   WND_DIR             56746 non-null  int32                          
 3   WND_SPEED           56746 non-null  float64                        
 4   RH                  56746 non-null  float64                        
 5   COMED_MW            56746 non-null  float64                        
 6   YEAR                56746 non-null  int32                          
 7   GDP                 56746 non-null  int64                          
 8   POPULATION          56746 non-null  int32                          
 9   DATE_CENTRAL        56746 non-null  datetime64[ns, America/Chicago]
 10  WEEKDAY_Fr

In [30]:
df.describe()

,TMP,WND_DIR,WND_SPEED,RH,COMED_MW,YEAR,GDP,POPULATION,HOUR,MONTHDAY,DAYOFYEAR,MONTHDAYRATIO,epoch,MIN_TEMP,MAX_TEMP
count,56746.000000,56746.000000,56746.000000,56746.000000,56746.000000,56746.000000,5.674600e+04,5.674600e+04,56746.000000,56746.000000,56746.000000,56746.000000,5.674600e+04,56746.000000,56746.000000
mean,51.021064,196.279914,10.482139,66.978651,11461.754661,2014.012406,8.739529e+06,2.457316e+07,11.696084,15.747224,182.534082,0.517034,1.404640e+09,43.131490,58.633672
std,20.853165,100.181709,4.918159,17.084137,2309.978674,1.998613,6.217311e+04,3.852406e+05,6.896085,8.800112,106.431708,0.288517,6.374871e+07,19.409248,21.279131
min,-16.060000,10.000000,3.355350,14.323454,7263.000000,2011.000000,8.646000e+06,2.410300e+07,0.000000,1.000000,1.000000,0.032258,1.293865e+09,-16.060000,-2.020000
25%,35.060000,110.000000,6.934390,54.823569,9829.000000,2012.000000,8.677000e+06,2.418900e+07,6.000000,8.000000,90.000000,0.266667,1.349731e+09,28.940000,41.000000
50%,51.080000,210.000000,9.171290,68.103793,11187.000000,2014.000000,8.739000e+06,2.453600e+07,12.000000,16.000000,181.000000,0.516129,1.404610e+09,42.980000,60.080000
75%,68.000000,280.000000,13.868780,80.303848,12553.000000,2016.000000,8.801000e+06,2.505700e+07,18.000000,23.000000,276.000000,0.774194,1.459673e+09,60.080000,77.000000
max,102.920000,360.000000,42.501100,100.000000,23753.000000,2018.000000,8.864000e+06,2.532600e+07,23.000000,31.000000,366.000000,1.000000,1.514765e+09,82.940000,102.920000


<h2>Save Dataframe</h2>

In [31]:
df.to_parquet('data/cleaned_data.parquet')